<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars/sem_6/student_teacher_dist.png" height="300" width="600"> 

# <center> R для тервера и матстата. <br>  <br> Домашка номер два! </center>

Данный ноутбук является домашкой по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2020). Автор ноутбука - [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/r_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

----------------------


__Краткий брифинг:__


* __Дедлайн:__ __31.05.20__
* Первая домашка затрагивает то, что мы смотрели на первых двух парах.
* Тетрадка делится на две части: __рыбёшки (55 баллов) и кит (45 баллов).__
* Часть задач нужно решить не только с помощью симуляций, но и вручную.
* Ваша команда пытается решить как можно больше задач и присылает их мне.
* Если очень хочется, можем устроить после всех дедлайнов серию из пар с разбором домашек. 
* Свои решения постараюсь выложить после дедлайна.
* __WARNING:__ [на страничке курса](https://github.com/FUlyankin/r_probability) есть решение домашек прошлого года. Ими можно пользоваться. Там много полезного и интересного! А ещё не стесняйтесь задавать вопросы :) 

---------------------------

* __Как оформлять домашки:__ 
    1. Компьютерные домашки оформляем либо в связке R + LaTeX, либо в юпитерских тетрадках (если хочется новых технологий), либо в связке R + Markdown. Все шаблоны для этого есть в репозитории. Пожалуйста не нужно присылать мне просто тупой код с выводами в виде комментариев, написанных через решёточки. Я от этого могу сойти с ума. 
    2. Ручные решение можно оформлять двумя способами: либо вбивать их в файл вместе с кодом. Благо в Markdown, юпитере и других местах поддерживаются теховские формулу. Либо аккуратно оформляйте их на бумаге и присылайте свёрстанную pdf-ку. Не надо кидать мне фотки по одной. Я от этого тоже сойду с ума. Вариант с вбиванием в основной файл - предпочтительнее. 
    
* На странице курса висит [видео-инструкция по оформлению.](https://yadi.sk/i/Pxp_pByP6Em9-A)

---------------------------

Ближе к делу. С чего начинается любой скрипт? Правильно! С подгрузки пакетов :) 

In [2]:
library("ggplot2")  # Пакет для красивых графиков 
library("reshape2")

library('dplyr')  # пакет для работы с таблицами 
library('rio')


library("grid")     # Пакет для субплотов

# Отрегулируем размер картинок, которые будут выдаваться в нашей тетрадке
library('repr')
options(repr.plot.width=4, repr.plot.height=3)

# Внимание! Если вы делаете дз в Rstudio, то вам не нужны пакеты grid, repr и т.п.
# Вам нужен только пакет ggplot2! 

Попробуйте самостоятельно реализовывать все критерии и доверительные интервалы в виде функций. На вход в эти функции должны попадать данные, на выходе должны оказываться разные полезности и результаты. Например, вот так можно было бы попробовать написать критерий для проверки гипотезы о доли. 

In [3]:
z.prop = function(m, n, p0, alpha=0.05, alternative="two-sided"){
  
  p_hat = m/n                       # оценка вероятности 
  sd_p = sqrt(p_hat*(1-p_hat)/n)    # дисперсия оценки
  z_stat = (p_hat - p0)/sd_p        # наблюдаемое значение

  # В зависимости от типа альтернативы делаем расчёты
  if(alternative == 'two-sided'){
      p_val <- 2*pnorm(-abs(z_stat))
      z_crit <- qnorm(1-alpha/2)
  }
  if(alternative == 'lower'){
      p_val <- pnorm(-abs(z_stat))
      z_crit <- qnorm(alpha)
  }
  if(alternative == 'greater'){
      p_val <- 1 - pnorm(abs(z_stat))
      z_crit <- qnorm(1-alpha/2)
  }
    
  # Возвращаем результаты проверки гипотезы в виде списка
  return( list(estimate = p_hat, 
               z_crit = z_crit,
               z_stat = z_stat,
               p_val = p_val,
               alternative = alternative))
}

# вот так можно к чему-нибудь применить функцию 
result <- z.prop(52, 100, 0.5, alpha=0.01, alternative = 'greater')

result  # вот так можно посмотреть на всё, что мы в неё запихнули

$estimate
[1] 0.52

$z_crit
[1] 2.575829

$z_stat
[1] 0.4003204

$p_val
[1] 0.3444603

$alternative
[1] "greater"

In [4]:
result$p_val # можно по именам из листа доставать разные циферки :3

[1] 0.3444603

Если кишка тонка писать свои функции, пользуйтесь готовыми. Но имейте в виду, что не все критерии, которые понадобятся вам в этой тетрадке, реализованы в R в виде одной команды. 

Если есть желание, можете снабдить выдачу картинками. В [тетрадке с лекции](https://nbviewer.jupyter.org/github/FUlyankin/r_probability/blob/master/end_seminars_2019/sem_6/6.%20Гипотезы%20о%20долях%2C%20средних%20и%20дисперсиях.ipynb) для этого написано несколько функций. Вы можете усовершенствовать их. Не путайте картинки местами! Например, не вздумайте использовать картинку для нормального распределения для t-критерия! Если код в ggplot кажется вам тяжёлым, можете попробовать рисовать картинки, [как во втором ответе тут.](https://stackoverflow.com/questions/36508020/can-r-visualize-the-t-test-or-other-hypothesis-test-results)

----------------------------

### [5]  Задачка 1

Аналитесса Аня наугад поймала $400$ покемонов-девочек и $100$ покемонов-мальчиков. Среди девочек $250$ оказались ядовитыми, среди мальчиков $60$. 

__[0.5]__ Методом моментов найдите точечную оценку для доли ядовитых покемонов среди мальчиков. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  Peace is a lie, there is only passion.

__[0.5]__ Найдите точечную оценку для дисперсии $X_i$, где $X_i$ - инекатор ядовитости $i$-го покемона-мальчика.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════   Through passion, I gain strength.

__[0.5]__ Постройте $95\%$ доверительный интервал для ядовитых покемонов-мальчиков. Какая у него получилась длина? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════   Through strength, I gain power.

__[0.5]__ Постройте $95\%$ доверительный интервал для ядовитых покемонов-девочек. Какая у него получилась длина? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════   Through power, I gain victory.

__[1]__ Постройте $95\%$ доверительный интервал для разницы долей ядовитых покемонов среди девочек и мальчиков. Какой длины он оказался? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  Through victory, my chains are broken.

__[1]__ Проверьте на уровне значимость $5\%$, что доля ядовитых мальчиков совпадает с долей ядовитых девочек. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  The Force shall free me.

__[1]__ Нужно ли предположение о нормальности $X_i$ и $Y_i$ для решения предыдущих пунктов? Почему? 

### [2]  Задачка 2

__[1]__ Аналитесса Аня пытается выяснить какая доля жителей Москвы любит кофе. Она хочет построить оценку так, чтобы ширина $95\%$ доверительного интервала для неё оказалась не больше $0.1$. Сколько наблюдений Ане необходисо собрать для достижения такой точности? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ скибиди па!

__[1]__ Аналитик Богдан проговорился Ане, что в прошлом году уже опрашивал так людей. У него оценка доли получилась в районе $0.7$. Аня верит Богдану как себе. Сколько наблюдений ей надо собрать для достижения той же точности с учётом этой информации. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ ратата!

### [6] [Задачка 3](https://nplus1.ru/news/2018/05/11/mice-trolley?utm_source=Telegram&utm_medium=Social) 

Для изучения аспектов процесса принятия моральных решений психологи уже много лет используют этические дилеммы, с помощью которых оценивают действия людей в гипотетических ситуациях. Один из самых известных примеров — это [проблема вагонетки,](https://ru.wikipedia.org/wiki/Проблема_вагонетки) в которой необходимо принять решение о том, стоит ли пожертвовать одним человеком для спасения пятерых.

Бельгийские психологи воплотили дилемму в реальную жизнь. Участники эксперимента должны были выбрать, ударить током пять мышей или одну мышь. Эксперимент проходил следующим образом. Участника сажали перед двумя клетками, в одной из которых сидели пять мышей, а в другой — одна. Перед клетками стоял ноутбук с 20-секундным таймером: участникам сообщили, что по истечении этих 20 секунд в клетку с пятью мышами пустят ток, и мыши получат не смертельный, но очень болезненный удар. Пока время идет, участник может нажать на кнопку: в этом случае ток пустят по клетке с одной мышью. В исследовании использовали живых мышей; следует, однако, уточнить, что удары тока были ненастоящими: сразу же после «удара» участников сопроводили в отдельную комнату, где разъяснили им, что мыши в полном порядке и током их не били (об этом заранее догадались только 12 участников). В решении реальной проблемы вагонетки приняли участие 192 человека, а еще 83 решали такую же задачку, но гипотетическую (на бумаге). Все участники также прошли онлайн-опросы, в ходе которых учёные собрали о респондентах кучу дополнительной информации. 

[В таблице](https://yadi.sk/i/ISGOdtr43Xq7bp) `mouse.csv` лежит информация о том, как прошёл эксперимент. Нас будут интересовать столбцы: 

* STUDY: какую проблему вагонетки решал человек 1 если реальную, 2 если на бумаге
* AGE: возраст респондента 
* GENDER: пол респондента
* DECISION: решение дилеммы (1 - жать на кнопу, 0 - не жать) 
* RT: время, которое респондент потратил, чтобы нажать на кнопку 

Подробное описание данных, сами данные и даже код на R, использованный при оценивании моделей, можно найти в [репозитории исследования.](https://osf.io/kvb99/) В статье авторы строили кучу логистических регрессий, чтобы очистить эффект от психологических особенностей респондентов. Этому добру вас будут учить на эконометрике, мы пойдём простым и не совсем правильным путём и будем проверять гипотезы в лоб. 

In [ ]:
df = import('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/2019/mouse.csv')
df = df[c('STUDY', 'AGE', 'GENDER', 'RT')]
head(df)

__Чётко обосновывайте тот критерий, который вы используете для теста гипотезы. Выписывайте для него все предпосылки.__ 

__[1]__ Попытаемся ответить на тот же вопрос, что и исследователи. В какой из двух ситуаций респонденты чащи принимают решение нажать кнопку: в реальной или на бумаге?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сккккрааа!

__[1]__ Правда ли, что женщины и мужчины принимают решению о нажатии на кнопку одинаково часто? А на бумаге?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ эщкере

__[1]__ Правда ли, что женщины намного кровожаднее мужчин и жмут на кнопку гораздо быстрее?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ пау! пау! пау!

__[1]__ Правда ли, что дисперсия времени для женщин равна дисперсии времени для мужчин? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ ррррааа!

__[1]__ Можем ли мы воспользоваться информацией о равенсве дисперсий для теста на равенство средних? Нужно ли делать коррекцию для уровня значимости? Если да, воспользуйтесь! 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆  пррррррр

__[1]__ Какая проблема при использовании точного критерия, в случае разных дисперсий возникает на практике? 

__Ответ:__ 

Все гипотезы тестируйте на уровне значимости $0.05$, не забывайте выводить $p$-значения. Кстати говоря, обратите внимание, что слово Задачка выделено синим. Это гиперссылка на статью на N+1. 

<img src="https://pp.userapi.com/c852220/v852220032/14dce1/r1zXBIig4B8.jpg" height="300" width="600"> 

### [2]  Задачка 4

__Чётко выпишите проверяемую гипотезу. Чётко обоснуйте выбранный для её проверки критерий, выпишите все предпосылки, которые вы используете.__

Ежегодно более $200000$ людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет $525$ баллов, стандартное отклонение — $100$ баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — $541.4$. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости $0.05$ нулевая гипотеза? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ бррря

### [2] Задачка 5 

__Чётко выпишите проверяемую гипотезу. Чётко обоснуйте выбранный для её проверки критерий, выпишите все предпосылки, которые вы используете.__

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало $50$ испытуемых. Каждый из них разговаривал с рекрутером; в конце $34$ из $50$ бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания $10$ из $34$ испытуемых экспериментальной группы и $4$ из $16$ испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно $4.4\%$. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости ($p-$значение) при альтернативе заразительности зевоты.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ як! як! як!

### [3+] Задачка 6

Эконооооом. [В табличке](https://yadi.sk/d/uT47H5783Xq4jM) `ekanam_data.csv` лежат данные про эконоооооом. [В вспомогательнй табличке](https://yadi.sk/d/MnJnluID3Xq4oL) лежит описание переменных из таблички про эконооооом. Нас будет интересовать не вся таблица, а вполне конкретные столбцы. 

В столбце `year` лежит год набора. В столбце `kozko` лежит массив из нулей и единиц. Единица означает, что лектором по матану был Козко. В столбце `target_1` лежит единичка, если человек дожил до конца первого курса. В столбце `target_2` лежит единица, если человек дожил до конца второго курса. 

In [ ]:
df = import('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/2019/ekanam_data.csv')
head(df)

__Чётко обосновывайте тот критерий, который вы используете для теста гипотезы. Выписывайте для него все предпосылки.__ 


__[1]__ Поговаривают, что Козко тот ещё зверь. Испокон веков, среди студентов эконома ходит миф, что когда матан читает Козко, с первого курса отчисляют больше людей. Когда матан читает Чирский, людей отчисляют меньше. Проверьте гипотезу о том, что при Козко с первого курса отчисляют больше людей. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ бррря

__[1]__ Проверьте гипотезу о том, что с первых двух курсов при Козко отчисляют больше людей, чем при Чирском.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ антихайп

__[1]__ Сделайте парочку осмысленных и глубоких выводов о том, что вы узнали выше. Как думаете, корректно ли мы проверяем эту гипотезу или что-то мы делаем не так? 

__[бонусные 5]__ Подумайте, какие ещё интересные гипотезы можно было бы проверить по экономовскому датасету. Проверьте их. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ скрррааа па па ка ка ка скидики па па энд э пу пу дррр бум скьа ду ду ку ку тун тун пум пум

__[бонусные 2]__ Что означает высказывание __скрррааа па па ка ка ка скидики па па энд э пу пу дррр бум скьа ду ду ку ку тун тун пум пум__? Откуда оно?

### [4] Задачка 7

Оборудование часто грешит и работает криво. Для каждого станка есть своя допустимая степень кривизны. Например, [в табличке](https://yadi.sk/i/bsjOHeOO3XaTN9) `gear.csv` лежат данные о производстве шестерёнок на станке. Первая колонка это диаметр, произведённой шестерни. Вторая колонка это то, к какой серии относится шестерня. 

Ясное дело, что станок не всегда делает диаметр равным $1$. У него есть некоторая погрешность. Будем считать, что станок годный, если дисперсия диаметра выпускаемых им шестерёнок не превышает $0.01$. Правда ли, что станок годный?

In [ ]:
df = import('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/gear.csv')
head(df)

__[2]__ Станок из задачки произвёл по очереди $10$ батчей из деталей. Понятное дело, что со временем со станком могут происходить проблемы, погрешность работы может искажаться, и из-за этого возникает необходимость подкрутить его. 

In [ ]:
unique(df$BATCH)

### ╰( ͡° ͜ʖ ͡° )つ──☆ Я изгоняю тебя Саруман, как яд изгоняют из раны.

__[2]__ Чтобы понимать, когда подкручивать, на станок нужно настроить какой-то тригер, который будет в случае чего срабатывать и требовать дополнительной проверки станка. Опишите, какую процедуру вы бы для этого использовали. Правда ли, что для $10$ батчей, которые произвёл станок, наш алерт ни разу не сработает? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ Ждите меня с первым лучом солнца, я приду на 5-ый день, с востока.

### [3] Задачка 8

Во время ЧЕ по футболу 2008 года и ЧМ 2010 года [Осьминог Пауль](https://ru.wikipedia.org/wiki/Пауль_(осьминог))занимался прогнозированием побед (после он ушёл в финансовую аналитику и IB). Осьминог дал верные прогнозы в $12$ случаях из $14$. Если предположить, что Пауль выбирает победителя наугад, какова вероятность получить 12 верных прогнозов из 14? 

Можно ли утверждать, что Осьминог обладает даром предвидения? Используйте для проверки гипотезы асиптотический критерий на основе нормального распределения. Каким получилось p-значение? 

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆ Духовной жаждою томим, В пустыне мрачной я влачился

__[бонусные 10]__ На основе распределения Бернулли выведите свой собственный критерий для строительства доверительных интервалов и проверки гипотез. Примените его к этой задачке. 

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆ И шестекрылый серафим на перепутье мне явился 
# Тут нужны и код и ручное решение

### [5] Задачка 9

Предположим, что все выпущенные танки имеют порядковый номер. От самого первого выпущенного танка, имеющего номер $1$, до самого последнего танка, имеющего номер $n$. В бою удалось подбить танки с номерами $8$, $12$ и $10$. постройте с помощью [метода моментов оценку количества танков.](https://www.youtube.com/watch?v=AGRhbYmg_fA)

__Подсказка:__ Если равновероятно выбирать от $1$ до $q$, то чему равно математическое ожидание? [Честный расчёт](https://www.overleaf.com/project/5e6f443c9f8ad70001cf8399) ищи тут в _вопросах из чата._

__Примечание:__ Незадолго до высадки союзников в Нормандии, 6 июня 1944 года, [в распоряжении союзников было всего два немецких танка «Пантера V».](https://en.wikipedia.org/wiki/German_tank_problem) По серийным номерам на шасси танков союзники оценили выпуск в феврале 1944 в 270 танков. Фактический выпуск «Пантер V» согласно немецким документам в феврале 1944 составил 276 танков. 

Следующие пункты решайте руками на бумажке. 

__[0.5]__ Найдите $E(X_i)$ и оценку метода моментов. Я не наркоман, я действительно выше дал ссылки на решение этого пункта. Воспроизведите его и осознайте. Особенно честный расчёт математического ожидания. Будет ли оценка несмещённой и состоятельной?  

__[0.5]__ Найдите $Var(X_i)$.  Вам понадобится тут формула для суммы квадратов первых $n$ чисел. Можете не доказывать её. Но для общей эрудиции посмотрите [одно из доказательств через биномиальные коэффициенты.](https://www.coursera.org/learn/kombinatorika-dlya-nachinayushchikh/lecture/v78nH/sliedstviie-no3-iz-posliedniegho-tozhdiestva) 

__[0.5]__ Найдите асимптотическое распределение $\bar x$ для текущей задачи. 

In [ ]:
# Что все эти аутсайдеры

__[0.5]__ Найдите асимптотическое распределение для оценки метода моментов на число танков. Постройте для числа танков $95\%$ доверительный интервал. 

In [ ]:
# Дружно нюхают дым с моего ствола

__[1]__ Найдите для оценки информацию Фишера. Можно ли сделать из неравенство Рао-Фреше-Крамера вывод о том, что наша оценка эффективна? 

In [ ]:
# Пепел и пекло - просто ад

__[0.5]__ Найдите $MSE$ оценки. 

In [ ]:
# Но адреналин прёт - просто вау

__[1]__ Представим себе, что мы хотим использовать в качестве оценки параметра $q$ оценку вида $k \cdot \hat q^{MM}$, где $\hat q^{MM}$ - оценка, полученная методом моментов. Найдите $k$, которое минимизирует $MSE$. Какими при этом окажутся смещение и разброс? 

In [ ]:
# Я выберу сам - быть мне царём горы тут

__[0.5]__  Предположим, что мы для получения оценки пытаемся минимизировать функционал ошибки: 

$$
L(\hat q) = (\hat q - \bar x)^2 + \lambda \cdot \hat q \to \min_q
$$

При каких значениях $\lambda$ он совпадет с оценкой метода моментов? При каких значениях $\lambda$ он совпадает с оценокой, минимизирующей $MSE$? Объясните в каких ситуациях имеет смысл использовать оценку, полученную минимизацией $MSE$? 

In [ ]:
# Или тлеющим остовом

__[бонусные 30]__ Проанализируйте оценку $\hat q = \max(X_1, X_2, X_3)$. Найдите её распределение, смещение, дисперсию, MSE. Найдите как именно надо скорректировать оценку, чтобы она стала несмещённой. Найдите как именно надо скорректировать оценку, чтобы минимизировать $MSE$. Будет ли итоговое $MSE$ больше, чем у оценки полученной в предыдущих пунктах с помощью среднего? Объясните почему так происходит с точки зрения интуиции. Как думаете, какую функцию потерь нужно минимизировать, чтобы получить такую оценку? 

__Примечание:__ вроде бы этот пункт решаемый, но надо будет повозиться с биномиальными и полиномиальными коэффициентами при поиске распределения максимума. 

In [ ]:
# ▅███████ ]▄▄▄▄▄▄▄ 
# █████████▅▄▃ 
# Il███████████████████] 
# ◥⊙▲⊙▲⊙▲⊙▲⊙▲⊙▲⊙◤

### [3] Задачка 10 

Таня всё ещё всерьёз увлечена лампочками. Она решила изучить их срок службы. Время службы лампы (в часах) имеет показательное распределение $Exp(\lambda)$. Таня уже испытала $100$ лампочек. Оказалось, что в среднем одна лампочка служит $0.52$ года. 

__[1.5]__ Пусть $g(\lambda)$ это математическое ожидание срока службы одной лампочки. Найдите эту функцию. Пусть $\hat \lambda$ - это оценка метода моментов для $\lambda$. Какое распределение имеет $g(\hat \lambda)$? Найдите параметры этого распределения. Постройте с помощью дельта-метода $95\%$ доверительный интервал для $g(\lambda)$. Все вычисления приложите к решению. 

In [ ]:
# ваш код 

__[1.5]__ Таню заинтересовала ещё и дисперсия! Проделайте для неё все ровно то же самое. 

In [ ]:
# ваш код 

### [10] Задачка 11

Нам с вами доводилось много раз проверять гипотезу о равенстве средних. Для этого мы делали следущие шаги: 

1. Собирали выборки $X_1, \ldots, X_n$ и $Y_1, \ldots, Y_n$;
2. Находили значение статистики

$$
z_{obs} = \frac{\bar x - \bar y}{\sqrt{\frac{s_x^2}{n_x} + \frac{s_y^2}{n_y}}};
$$

3. Говорили, что по ЦПТ $z_{obs} \overset{asy}{\sim} N(0,1);$
4. Находили критическое значение $z_{1 - \frac{\alpha}{2}}$;
5. Если мы видели, что $|z_{obs}| <  z_{1 - \frac{\alpha}{2}}$, мы говорили, что гипотезв не отвергается. 

Ту же самую гипотезу можно попробовать проверить с помощью немного иной процедуры, основанной на доверительных интервалах: 

1. Найдём $\bar x$ и $\bar y$;
2. Пользуясь ЦПТ и зная, что $\bar x \overset{asy}{\sim} N \left(\mu_1,\frac{s^2_x}{n_x} \right)$ и $\bar y \overset{asy}{\sim} N\left(\mu_2,\frac{s^2_y}{n_y}\right)$ построим для $\mu_1$ и $\mu_2$ доверительные интервалы;
3. Если доверительные интервалы пересеклись, говорим, что гипотеза не отвергается. 

Вроде бы вторая процедура выглядит довольно естественно, однако ей никто не пользуется. Дело в том, что для одинаковых ошибок первого рода, $\alpha$, ошибка второго рода, $\beta$, для процедуры, основанной на доверительных интервалах, окажется выше. Давайте для простоты будем дальше предполагать, что $\bar x > \bar y$. Если это не так, просто будем менять обозначения. 

__[5]__ Найдите ошибку второго рода для первой процедуры. Запишите её через функцию распределения стандартной нормальной величины, $\Phi(x)$.

__[5]__ Выпишите доверительный интервал для $\bar x$ и для $\bar y$.  Правая граница интервала для $\bar x$ должна оказаться правее левой границы для $\bar y$. В таком случае гипотеза не отвергается. Выпишите это неравенство. Попытайтесь выразить из него значение $z_{1 - \frac{\alpha}{2}}$. Сравните получившуюся статистку со статистикой из первого пункта. Найдит едля неё ошибку второго рода, запишите её через функцию распределения стандартной нормальной величины, $\Phi(x)$.

__[5]__ Выпишите для обеих процедур функции мощности и сравните их друг с другом. Постройте в R для них графики.

__[5]__ Постройте функции мощности для описанных процедур с помощью симуляций. Сравните их с теоретическими. Сделайте выводы. 

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆  ваше решение

### [10] Задачка 12

Ещё одна задачка на мощности! Давайте проведём модельный эксперимент. Сгенерируйте $50$ выборок из нормального распределения $N(1,1)$ и ещё $150$ выборок из нормального распределения $N(0,1)$. Объём выборок должен составить $n$. Для каждой из выборок проверьте гипотезу о равенстве нулю среднего против двусторонней альтернативы с помощью теста Стьюдента. Сделайте это тремя способами.

1. Без корректировки на множественную проверку гипотез 
2. С поправкой Бонфферони 
3. С поправкоф методом Холма 
4. С поправкой методом Бенджамини-Хохберга

Будем считать число неверно отвергнутых (или не отвергнутых гипотез). Дальше под ошибками первого и второго рода имеется в виду именно это. 

__[10]__ Подсчитайте количество ошибок первого и второго рода, которое в среднем совершается при $n=20$ каждой из этих процедур. Где оно оказалось больше?

__[10]__ Постройте для всех трёх при разных $n$ графики с оценками для ошибок первого и второго рода. Какая из процедур оказалась мощнее? 

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆  ваше решение

##  [45+] Кит-рисёрчер

**Добро пожаловать в Кита. Кит — это большая рисёрчерская задача с огромным количеством вопросов. В этот раз в нём не будет ничего сложного. Просто очень много вопросов, картинок и выводов о состоянии онлайн-магазина.** 

Чтобы держать руку на пульсе бизнеса, нужно строить довольно много продуктовых метрик. Более того, нужно понимать насколько адекватно эти метрики построены и насколько сильно они могут колебаться.

__Словарик:__ 

* __retention__ - возвращаемость пользоватяля к использованию продукта через $n$ дней (через день, два, неделю и тд) 
* __RPU__ - выручка на юзера, купившего товар (revenue per user) 
* __DAU__ - дневная активная аудитория (daily active user) 
* __LTV__ - совокупная прибыль, полученная за всё время сотрудничества с клиентом (Lifetime Value) 



Анализировать будем данные по продажам в [гугловом мерчендайз магазине.](https://www.kaggle.com/c/ga-customer-revenue-prediction) Скачайте [табличку с данными с гита.](https://github.com/FUlyankin/r_probability/blob/master/data/marketing_log.csv)

- Для решения задачи используйте библиотеку `dplyr` для работы с таблицами. Никаких циклов! Только `groupby`, трубочки, фильтры и тп. 
- В текстовых полях с выводами не надо писать огромные абзацы текста. Пишите тезисно. 

In [ ]:
df = read.csv('../../data/marketing_log.csv', sep='\t')
head(df)

__[2]__ Сколько людей пользовались магазином в среднем в течение дня/месяца? Постройте дневной график с суммарным числом пользователей, заходивших на сайт. Постройте месячный график. 

In [ ]:
# Призываю команду к коду

__[2]__ Проинтепретируйте график: 

* Наблюдается ли на нём сезонность? Какая она? 
* Когда в посещении сайта произошёл резкий всплеск? Как думаете с чем это связано? 
* Хороша ли эта метрика для оценки бизнеса? В чём её минусы?  

__Ответ:__

__[1] Постройте график с числом пользовательских сессий.__

In [ ]:
# Призываю команду к коду

__[1]__ Насколько он сильно отличается от графика с числом уникальных пользователей. Как думаете о чём это может сигнализировать?  

__Посмотрим на возвращаемость пользователей (retention).__ 

* `retention_1`  - вернулся ли игрок после **первого** дня с момента посещения магазина?
* `retention_7`  - вернулся ли игрок после **седьмого** дня с момента посещения магазина?

__[1]__ Заведите колонку: _дата первого появления юзера._

In [ ]:
# Призываю команду к коду

__[1]__ Посмотрите на то, как распределено число дней между текуим и первым посещением. Для этого постройте гистограмму. 

In [ ]:
# Призываю команду к коду

__[1]__ Что интересного на ней вы видите? 

__[1]__ Создайте колонку с разностью в днях между первым и последним посещениями. Назовите её `daysFromFirstVisit`.

In [ ]:
# Призываю команду к коду

__[2]__ Посчитаем `retention_7`. Если `daysFromFirstVisit` оказывается больше $7$, значит человек вернулся более, чем через $7$ дней после первого посещения. Доля таких людей для каждого дня и есть `retention_7`. Найдите её для каждого дня и постройте картинку. 

In [ ]:
# Призываю команду к коду

__[2]__ Есть много точечных оценок доли людей, которая возвращается для каждой даты. Но этого мало, каждая точка строится по разному числу наблюдений и нам хотелось бы понимать насколько оценка точная. Возвращаемость - это доля. Значит можно построить для неё $95\%$ доверителльный интервал с помощью ЦПТ:

$$
\hat p \pm z_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}} 
$$

In [ ]:
# Призываю команду к коду

__[2]__ Постройте для семидневной возвращаемости $95\%$ доверительный интервал Уилсона.

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{
z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [ ]:
# Призываю команду к коду

__[1]__ В каких ситуациях он обычно используется? Шире он получился или уже? Как думаете, почему? 

__[3]__ Финальный заход на анализ возвращаемости. [Построим heatmap](http://www.sthda.com/english/wiki/ggplot2-quick-correlation-matrix-heatmap-r-software-and-data-visualization) с возвращаемостью по когортам. За когорту будем рассматривать людей, пришедших в течение конкретного месяца. Должна получиться примерно такая картинка: 

![](https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars_2020/HW/retention.png) 


По оси $y$ откладывайте месяц первого прихода. По оси $x$ откладывается месяц, на который мы смотрим __retention.__ Например, клетка на пересечении да `2017-01` по $x$ и `2016-08` по $y$ - число людей, которые вернулис на сайт в январе $2017$ и при этом зарегистрировались в августе $2016$. 

In [ ]:
# Призываю команду к коду

__[1]__ Какие выводы вы можете сделать о возврате пользователей после всех манипуляций с retention? 

__Проанализируем продажи.__

__[1]__ Какой процент людей, зашедших на сайт делает покупки?

In [ ]:
# Призываю команду к коду

__[1]__ После какого посещения они делают свою первую покупку? Постройте гистограмму.

In [ ]:
# Призываю команду к коду

__[1]__ Сколько покупок делает один пользователь? Постройте гистограмму числа покупок.

In [ ]:
# Призываю команду к коду

__[1]__ Какие выводы о покупках можно сделать на основе построенных картинок? 

__[1]__ Отберите в маленькую табличку все записи лога, которые соотвествуют покупке. 

In [ ]:
# Призываю команду к коду

__[1]__ Посмотрите на то как выглядит распределение стоимости покупок.

In [ ]:
# Призываю команду к коду

В данных явно есть выбросы. Видно, что у распределения очень длинный хвост. Нам хотелось бы построить для средней выручки с пользователя доверительный интервал, но выбросы будут мешать нам использовать ЦПТ, которая предполагает что все случайные величины, участвующие в поиске среднего имеют одинаковое распределение, и аномалии не встречаются. 

__[1]__ Давайте избавимся от выбросов. Для этого найдите $95\%$ квантиль и срежте все наблюдения, где выручка больше него. Снова постройте гистограмму. 

In [ ]:
# Призываю команду к коду

__[1]__ Посчитайте по выручке среднее, стандартное отклонение и число наблюдений для каждого месяца.

In [ ]:
# Призываю команду к коду

__[1]__ Наконец, воспользуемся формулой для строительства доверительного интервала для среднего: 

$$
\hat \mu \pm z_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat{\sigma}^2}{n}}.
$$

Изобразите месячную динамику средней RPU на картинке. Нанесите на неё $95\%$ доверительный интервал. 

In [ ]:
# Призываю команду к коду

__[1]__ Какие выводы после проведённого анализа вы можете сделать?

__[2]__ Посчитайте сколько суммарно приносят денег разные пользователи в течение всего своего взаимодействия с магазином. Постройте для средне-месячного LTV такую же картинку как и для retention.

In [ ]:
# Призываю команду к коду

__[1]__ Какие выводы вы можете сделать? 

__[2]__ С каких устройств (колонка `deviceCategory`) покупки осуществляются чаще всего? Постройте столбиковую диаграмму для числа покупок с каждого типа девайска (десктоп, мобила, планшет). 

In [ ]:
# Призываю команду к коду

__[2]__ Как выручка распределена по устройствам? Постройте ящик с усами или виалончель, чтобы посмотреть на это. 

In [ ]:
# Призываю команду к коду

__[1]__ Какой вывод вы можете сделать на основе построеных картинок? 

__Ответ:__ 

In [ ]:
Призываю команду к ответу! 

__[2]__ Проверьте гипотезу о том, что число среднняя выручка с десктопа больше, чем с мобилы и планшета. Строго опишите ту процедуру, которую вы используете для проверки этой гипотезы. 

In [ ]:
# Призываю команду к коду

__[2]__ Постройте доверительный интервал для месячной средней выручки в разбивке по девайсам. Как думаете меняется ли во времени доходность разных девайсов или одни из них на протяжении всего периода приносят значимо больше денег, чем другие? 

In [ ]:
# Призываю команду к коду

__[бонусные 10]__ На какие ещё вещи вам было бы интересно посмотреть по текущему датасету? Если это возможно, изучите их. 

In [ ]:
# Призываю команду к коду

__[2]__ На основе всего проведённого анализа сделайте выводы о текущем состоянии магазина и дайте ему пару советов: 

__Выводы:__ 